## Librerias a utilizar

In [1]:
import csv
import pandas as pd
import xml.etree.ElementTree as ET
from os import scandir

## Extrayendo los datos

In [2]:
def getData():

  # Read the TXT file with the clasification of the user's names
  usersClasification = pd.read_csv('truth.txt', sep=':::', header=None, engine='python')
  usersClasification.columns = ["User Name", "Type", "Gender"]

  print("Users clasification: \n\n{}".format(usersClasification))

  # Path of the XML files
  path = 'pan19_author_profiling_training_es/'
  
  # Get all the names of the files in the path
  files = [arch.name for arch in scandir(path) if arch.is_file()]

  # Extract only the User name, without the file extension (.xml)
  userNames = []
  fileExtension = '.xml'
  for fileName in files:
    index = fileName.index(fileExtension)
    userNames.append(fileName[:index])
  
  filesContent = []
  key = ''
  tweetsPerUser = {}
  index = 0
  
  # Extract the data for each XML file and save it in a list (filesContent)
  for arch in scandir(path):
    if arch.is_file():
      try:
        # Read the info in each file
        tree = ET.parse('pan19_author_profiling_training_es/{}'.format(arch.name))
        root = tree.getroot()
        
        # Extract the info in each file
        filesContent = [i.text for i in root.iter('document')]
        
        # Build a dictionay (Key[User name] | Value[tweets of the user])
        key = userNames[index]
        tweetsPerUser.setdefault(key, filesContent)
        index += 1
      except:
        print("Error in the file: {}".format(arch.name))

  # Return a list with the clasification based on the User's names and a dictionary with the Tweets for each User
  return usersClasification, tweetsPerUser

## Clasificando a los usuarios

In [3]:
def doComparations(usersClasification, tweetsPerUser):
  finalClasification = {}
  for user in range(len(usersClasification)):
    for twitterUser in tweetsPerUser:
      if(usersClasification['User Name'][user] == twitterUser):
        # Key
        userName = twitterUser
        
        # Value 1
        tweets = tweetsPerUser[twitterUser] 
        
        # Value 2
        userType = usersClasification['Type'][user]
        
        # Value 3
        userGender = usersClasification['Gender'][user]
        
        finalClasification.setdefault(userName, [tweets, userType, userGender])
        
  return finalClasification

## Creando el archivo CSV

In [4]:
def buildCSV(finalClasification):

  # Build the CSV file
  with open('Users_Clasifiaction.csv', 'w', newline='',encoding="utf-8") as file:
    writer = csv.writer(file, delimiter='|')
    writer.writerow(["User", "Type", "Tweets"])
    for user in finalClasification:
      tweets = str(finalClasification[user][0])
      if(finalClasification[user][1] == "human"):
        writer.writerow([user, 1, tweets])
      else:
        writer.writerow([user, 0, tweets])

## Función principal

In [5]:
if __name__=='__main__':
  
  # Get the data of the files
  usersClasification, tweetsPerUser = getData()
  finalClasification = doComparations(usersClasification, tweetsPerUser)
  buildCSV(finalClasification)
  print("Ejecución exitosa")

Users clasification: 

                             User Name   Type Gender
0     acfdc9e43ed5efa10a903453261a3c12    bot    bot
1     40adbb05f96fdd96f767b5967458faf1    bot    bot
2      4aa2fb302140ec35cc6bc8a0d7d35f6    bot    bot
3     58db587d884d22afefbcd37aa26af458    bot    bot
4     dd9494d1bff7fa477cc03fea5294a510    bot    bot
...                                ...    ...    ...
2995   cc5432be0f89f59bb9a54a546a0a45b  human   male
2996  7388a7fda44ec828643c1e6a60d16a31  human   male
2997  72472a9c1db1190f972c0685c17e6967  human   male
2998  fad9f50742fbb4754ffea1faed468450  human   male
2999  de49aeb2e3b73a0204bd6f82bfa4b17d  human   male

[3000 rows x 3 columns]
Ejecución exitosa
